In [4]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
# Import necessary metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
# Import machine learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [6]:
#df = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\CIC_IDS_Merged.csv")
df1 = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\UNSW_NB15.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_16740\3018969767.py:2: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r"C:\Users\hp\Desktop\TFG\Datasets\UNSW_NB15.csv")


In [28]:
negative_columns = (df['Init Fwd Win Byts'] < 0).sum()
print(f"Negative values per column:\n{negative_columns}")

Negative values per column:
1925400


In [5]:
# Normalize the 'attack_cat' column directly
def normalize_label(label):
    if pd.isnull(label):
        return 'benign'  # Assign NaN values to 'benign'
    
    # Remove extra spaces and make the label lowercase
    label = str(label).strip().lower()
    
    # Dictionary for replacing and normalizing labels
    replacements = {
        'backdoors': 'backdoor',  # Merge 'backdoors' into 'backdoor'
        'shellcode': 'shellcode',
        'fuzzers': 'fuzzers',
        'reconnaissance': 'reconnaissance',
        'analysis': 'analysis',
        'worms': 'worms',
        'exploits': 'exploits',
        'dos': 'dos',  # Normalize DoS to dos
    }
    
    # Apply the replacement rules
    return replacements.get(label, label)

# Apply the normalization function directly to the 'attack_cat' column
df1['attack_cat'] = df1['attack_cat'].apply(normalize_label)


In [7]:

# Rename columns to match UNSW naming conventions
df = df.rename(columns={
    'Flow Duration': 'dur',
    'Dst Port': 'dsport',
    'Tot Fwd Pkts': 'Spkts',
    'Tot Bwd Pkts': 'Dpkts',
    'TotLen Fwd Pkts': 'sbytes',
    'TotLen Bwd Pkts': 'dbytes',
    'Fwd Pkt Len Mean': 'smeansz',
    'Bwd Pkt Len Mean': 'dmeansz',
    'Init Fwd Win Byts': 'swin',
    'Init Bwd Win Byts': 'dwin',
    'Fwd IAT Std': 'Sjit',
    'Bwd IAT Std': 'Djit',  # Align label name with UNSW
})

# Drop non-overlapping features
df = df.drop(columns=[
    'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
    'Fwd Header Len', 'Bwd Header Len', 'Active Mean', 'Idle Mean',  # CIC-specific
    'Label'  # Original CIC label (keep only 'is_attack' renamed to 'Label')
], errors='ignore')

# Ensure data type alignment
df['dbytes'] = df['dbytes'].astype(float)

In [21]:
print(df[['dur', 'Sjit', 'Djit', 'Fwd IAT Mean']].describe())

                dur          Sjit          Djit  Fwd IAT Mean
count  7.439296e+06  7.439296e+06  7.439296e+06  7.439296e+06
mean   1.111672e+07  1.945504e+06  9.909242e+05  2.486580e+06
std    4.740344e+08  2.753818e+08  4.549116e+06  1.255162e+08
min   -9.190110e+11  0.000000e+00  0.000000e+00 -3.407010e+11
25%    2.840000e+02  0.000000e+00  0.000000e+00  0.000000e+00
50%    1.087200e+04  0.000000e+00  0.000000e+00  4.605000e+02
75%    1.054092e+06  2.646096e+04  5.982377e+03  1.181196e+05
max    1.200000e+08  4.244739e+11  8.480000e+07  1.200000e+08


In [27]:
print(df1[['dur', 'Sjit', 'Djit', 'Sintpkt','Dintpkt']].describe())


                dur          Sjit          Djit       Sintpkt       Dintpkt
count  2.540043e+06  2.540043e+06  2.540043e+06  2.540043e+06  2.540043e+06
mean   6.587925e-01  1.589039e+03  7.300766e+02  1.933228e+02  7.882489e+01
std    1.392494e+01  1.691038e+04  3.438561e+03  2.779165e+03  1.433193e+03
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    1.037000e-03  0.000000e+00  0.000000e+00  9.000000e-03  6.000000e-03
50%    1.586100e-02  1.912483e+01  2.653536e+00  4.682620e-01  4.147540e-01
75%    2.145650e-01  4.137935e+02  6.350857e+01  7.351437e+00  6.202144e+00
max    8.786638e+03  1.483831e+06  7.812211e+05  8.437150e+04  5.948532e+04


In [9]:
# Convertir todas las features temporales de UNSW a microsegundos
df1['dur'] = df1['dur'] * 1_000_000      
df1['Sjit'] = df1['Sjit'] * 1000          
df1['Djit'] = df1['Djit'] * 1000          
df1['Sintpkt'] = df1['Sintpkt'] * 1000    
df1['Dintpkt'] = df1['Dintpkt'] * 1000   

In [17]:
df1 = df1.rename(columns={
    'dur': 'dur',
    'dsport': 'dsport',
    'Spkts': 'Spkts',
    'Dpkts': 'Dpkts',
    'sbytes': 'sbytes',
    'dbytes': 'dbytes',
    'smeansz': 'smeansz',
    'dmeansz': 'dmeansz',
    'swin': 'swin',
    'dwin': 'dwin',
    'Sjit': 'Sjit',
    'Djit': 'Djit',
    'Label':'is_attack'
})


# Drop non-overlapping features
df1 = df1.drop(columns=[
    'srcip','proto', 'dstip', 'sport', 'state', 'ct_state_ttl', 'ct_flw_http_mthd',
    'is_sm_ips_ports', 'ct_ftp_cmd', 'attack_cat'  # UNSW-specific
], errors='ignore')

# Ensure data type alignment
df1['is_attack'] = df1['is_attack'].astype(int)
# Calcular tasas y manejar divisiones por cero
df1['Flow Byts/s'] = np.where(
    df1['dur'] != 0,  # Evitar división por cero
    (df1['sbytes'] + df1['dbytes']) / (df1['dur'] / 1_000_000),  # B/s
    np.nan  # Asignar NaN si dur=0
)

df1['Flow Pkts/s'] = np.where(
    df1['dur'] != 0,
    (df1['Spkts'] + df1['Dpkts']) / (df1['dur'] / 1_000_000),  # Pkts/s
    np.nan
)

# Calcular Fwd IAT Mean y manejar Spkts + Dpkts = 0
df1['total_pkts'] = df1['Spkts'] + df1['Dpkts']
df1['Fwd IAT Mean'] = np.where(
    df1['total_pkts'] != 0,
    (df1['Sintpkt'] * df1['Spkts'] + df1['Dintpkt'] * df1['Dpkts']) / df1['total_pkts'],
    np.nan
)
common_features = [
    'dur', 'dsport', 'Spkts', 'Dpkts', 'sbytes', 'dbytes',
    'smeansz', 'dmeansz', 'swin', 'dwin', 'Sjit', 'Djit',
    'Flow Byts/s','Flow Pkts/s','Fwd IAT Mean','is_attack'
]
df = df[common_features]
df1 = df1[common_features]

In [25]:
df1 = df1.sample(frac=1, random_state=42).reset_index(drop=True) #Shuffle UNSW NB15 for randomness

In [35]:
# Convert dsport to numeric, coercing errors to NaN
df1['dsport'] = pd.to_numeric(df1['dsport'], errors='coerce')
# For CICIDS (df)
df['dur'] = df['dur'].astype('float64')  # Match UNSW's dtype
# For UNSW (df1)
df1['dbytes'] = df1['dbytes'].astype('float64')   # Match CICIDS' dtype
df1['smeansz'] = df1['smeansz'].astype('float64')
df1['dmeansz'] = df1['dmeansz'].astype('float64')

In [49]:
# Drop rows with NaN in dsport
df1 = df1.dropna(subset=['dsport'])

In [57]:
# For UNSW (df1)
df1['dsport'] = df1['dsport'].astype('int64')
# For UNSW (df1)
df1['is_attack'] = df1['is_attack'].astype('int64')

In [33]:
From this point we have the datasets cic ids and unsw ready to be merged and we remove the 

,dur,dsport,Spkts,Dpkts,sbytes,dbytes,smeansz,dmeansz,swin,dwin,Sjit,Djit,Flow Byts/s,Flow Pkts/s,Fwd IAT Mean,is_attack
0,285356.0,3260,6,2,986,86,164,43,255,255,3661562.382,0.000,3.756711e+03,28.035156,42803.900000,0
1,314311.0,6881,16,18,1540,1644,96,91,255,255,1385623.880,35106.383,1.013009e+04,108.173115,19619.907647,0
2,301180.0,38558,4,4,536,304,134,76,0,0,116927.883,123543.578,2.789030e+03,26.562189,85023.000000,0
3,9.0,53,2,0,114,0,57,0,0,0,0.000,0.000,1.266667e+07,222222.222222,9.000000,1
4,1079.0,53,2,2,146,178,73,89,0,0,0.000,0.000,3.002780e+05,3707.136237,8.000000,0


In [59]:
# Merge datasets
merged_df = pd.concat([df, df1], ignore_index=True)
# Save to CSV
merged_df.to_csv(r"C:\Users\hp\Desktop\TFG\Datasets\mergedDS.csv", index=False)

In [96]:
def map_unsw_attack(attack):
    if attack == "benign":
        return "Normal"
    elif attack == "dos":
        return "DoS"
    elif attack in ["reconnaissance", "fuzzers"]:
        return "Reconnaissance"
    elif attack == "exploits":
        return "Exploits"
    elif attack in ["backdoor", "worms", "shellcode"]:
        return "Infiltration"
    else:
        return attack  # Leaves 'generic' and 'analysis' unchanged

In [294]:
X = merged_df.drop(columns=["is_attack"])
y = merged_df["is_attack"]

In [296]:
# Split into train/test sets (stratify to handle imbalance)
X_train, X_test, y_train, y_test = train_test_split(
    X,  # or X if not scaled
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [300]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Fit the model to the training data
rf_model.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [302]:
# Make predictions on the test set
y_pred = rf_model.predict(X_test)
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)



Accuracy: 0.9769463086235871
Precision: 0.9695977796845372
Recall: 0.935610884982347
F1-Score: 0.9523011873202685
Classification Report:


NameError: name 'classification_report' is not defined